# Lab 11: Magnetic anomaly across a spreading ridge
---

## Submission

Please upload your completed notebook (renamed with your name) to Canvas as usual.

## Introduction

In this lab, we are going to write code to compute marine magnetic anomalies for trench-normal magnetic profiles. To do this, we will take advantage of a mathematical transformation that lets us derive the expression for a magnetic anomaly from the expression for a gravity anomaly. The magnetic anomalies in question are *total field anomalies*, meaning the observed (vector) magnitude of the magnetic field minus the vector magnitude of the reference field. For total field anomalies the component of the anomaly vector that we are measuring is the component in the direction of the reference field, where $\mathbf{F}$ is the reference field and $\mathbf{\Delta F}$ is the vector anomaly:

$$\Delta B = \mathbf{\hat{F}} \cdot \mathbf{\Delta F}\,.$$

Therefore, to compute the model anomalies we need to determine both the reference field vector and the model anomaly vector for a particular model of magnetization. We will be using the IGRF as the reference vector field.


## Computing magnetic anomalies

Write a function that will compute the magnetic anomalies along a profile across a spreading ridge, given the following inputs: Location of the ridge axis, azimuth of the profile, depth of the seafloor, thickness of the seafloor $T$, the (scalar) strength of magnetization of the seafloor $M$, and the spreading rate. This function requires several steps:

1. Use the ridge location and azimuth to compute the locations of points on the profile (use the provided function `track1()`). Use the data from Figure 1-29 to compute a synthetic profile. You will need to estimate the azimuth of the profile from a map, as that is not given in the text, and the same for the seafloor depth.

2. Use the magnetic timescale from Turcotte and Schubert (Table 1-4, p. 29, see variable `NORMAL_POLARITY` defined below for your convenience) along with the spreading rate to generate a magnetization model for each point on the profile. Assume that locations with normally magnetized seafloor have a magnetization vector of $+MT$ in the direction of the regional field, and locations with reverse magnetized seafloor have a magnetization vector of $-MT$ in the direction of the regional field. Make sure you are consistent about whether you mean the full spreading rate (separation rate of the plates) or the half spreading rate.

3. Compute the anomaly using the expression given below for a thin 2D strip of material. This is derived directly from a version of equation 5-106 for the gravity anomaly (Problem 5-15 in Turcotte and Schubert).

Try to approximate the magnetic profile along the East Pacific Rise (Figure 1-29(a)), using the spreading rate given in panel (c). Use $T = 1000$ m. Plot your magnetization model and profile. You will need to adjust the scalar value of $M$ to roughly match the observed magnitude of the anomalies (which are in microtesla, μT). **You must only compute one side of the profile, since we are assuming it is symmetric.**

Use the expression for the gravitational attraction of the same ribbon with surface density $\sigma$, where the edges of the ribbon are located at $(r_1, \theta_1)$ and $(r_2, \theta_2)$ relative to the observation point, as shown below. This model is for a thin ribbon, where thin means that the thickness of the ribbon is small compared to the water depth (the height of the measurement above the ribbon). See the figure below for definition of the symbols. The expression for the magnetic anomaly, from Blakely (1996), is

$$B_x = -2 C_m \left(\mathbf{M} \cdot \mathbf{\hat{n}}\right) \left[\hat{s}_x \log\frac{r_2}{r_1}-\hat{s}_z(\theta_1 - \theta_2)\right]$$

$$B_z = -2 C_m \left(\mathbf{M} \cdot \mathbf{\hat{n}}\right) \left[\hat{s}_z \log\frac{r_2}{r_1}+\hat{s}_x(\theta_1 - \theta_2)\right].$$

$\mathbf{M}$ is the magnetization vector. Angles are in radians. The constant $C_m = 10^{-7}$ for SI units ($\mathbf{B}$ in tesla, T). In this expression, $\mathbf{\hat{n}}$ is a vector normal to the face of the ribbon, and $\mathbf{\hat{s}}$ is a vector parallel to the width of the ribbon (the finite direction). In our case, $\mathbf{\hat{n}}$ points in the $-z$ direction, and $\mathbf{\hat{s}}$ points in the $+x$ direction, so the expressions simplify to

$$B_x = 2 C_m \left(\mathbf{M} \cdot \mathbf{\hat{z}}\right) \log\frac{r_2}{r_1}$$

$$B_z = 2 C_m \left(\mathbf{M} \cdot \mathbf{\hat{z}}\right) (\theta_1 - \theta_2)\,.$$

Note that the magnetization used here has units of magnetic moment per unit area.

<img src="https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_11/fig_9_14.png" width=500>

In [ ]:
import nvector
import numpy as np


def track1(origin_lat, origin_lon, azimuth, distance, npts):
    """Geographic tracks from starting point, azimuth, and range.
    
    Python version of MATLAB's function of the same name, albeit with
    fewer options (https://www.mathworks.com/help/map/ref/track1.html).
    Uses the WGS84 ellipsoid.
    
    Args:
        origin_lat (int or float): Starting latitude in degrees
        origin_lon (int or float): Starting longitude in degrees
        azimuth (int or float): Azimuth of track (degrees from N)
        distance (int or float): Total distance of track in km
        npts (int): Number of points in track
    
    Returns:
        Tuple containing (latlist, lonlist)
    """
    
    origin = nvector.FrameE().GeoPoint(latitude=origin_lat, longitude=origin_lon, degrees=True)
    sample_pts = np.linspace(0, distance * 1000, npts)  # Converting from km to m here
    
    lats = []
    lons = []
    for dist in sample_pts:
        pt = origin.displace(distance=dist, azimuth=azimuth, degrees=True)[0]
        lats.append(pt.latitude_deg)
        lons.append(pt.longitude_deg)
        
    return lats, lons


# Periods of NORMAL anomalies (starttime, endtime) in Ma
NORMAL_POLARITY = np.array([
    (0.00, 0.72),
    (0.91, 0.97),
    (1.65, 1.88),
    (2.06, 2.09),
    (2.45, 2.91),
    (2.98, 3.07),
    (3.17, 3.40),
    (3.87, 3.99),
    (4.12, 4.26),
    (4.41, 4.48),
    (4.79, 5.08),
])